## get data

In [9]:

from GOLDS import cleaning_data, Save_data_gold_transform
from BRONZE import savedataraw
from SILVER import Save_data_transform
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()
clean = cleaning_data()
data_raw = savedataraw()
data_silver= Save_data_transform()
data_gold =Save_data_gold_transform()

In [10]:
list =["flights", "airports", "airlines","zones"]
for data_type in list :
    data_raw.save_raw(data_type)
    data_silver.save_to_parquet(data_type)
    data_gold.save_data_to_parquet(data_type)

c:\Users\mdori\OneDrive\Documents\Flight_radar\BRflight_radar\GOLDS.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean= df_flights.applymap(lambda x: None if x == "N/A" else x)


In [11]:

if data_type== "flights" :
    flights= clean.clean_data(data_type)
elif data_type=="airports":
    airports=clean.clean_data(data_type)
elif data_type=="airlines":
    airlines=clean.clean_data(data_type)
elif data_type=="zones":
    zones=clean.clean_data(data_type)

In [12]:
data1 = "flights"
data2 = "airports"
distance= clean.clean_distance(data1 ,data2)
data_raw.save_distance (data1, data2) 
data_gold.save_distance_to_parquet(data1, data2)


In [15]:
from pyspark.sql import SparkSession
import pandas as pd
# Create a SparkSession
spark = SparkSession.builder.appName("Projet").getOrCreate()


In [16]:
from pyspark.sql import functions as F
from pyspark.sql.functions import max
from pyspark.sql.functions import*

## Read all data in spark

In [ ]:
#read flights
df_flights=spark.createDataFrame(clean.clean_data("flights"))
df_flights.show()

In [ ]:
 #read flights
df_flights =spark.createDataFrame(clean.clean_data("flights"))
# read airlines
df_airlines = spark.createDataFrame(clean.clean_data("airlines"))
# read airports   
df_airports = spark.createDataFrame(clean.clean_data("airports"))
# read Zones 
df_zones= spark.createDataFrame(clean.clean_data("zones"))
              
# read distances flight
df_distance=spark.createDataFrame(clean.clean_distance(data1 ,data2))  

## La compagnie avec le + de vols en cours

In [19]:
#read df_flights
df_flights.show()

+-------------+------------+------------+------------------------+-----------+---------+-------------+---------+-------------------+----------------------------+
|aircraft_code|airline_iata|airline_icao|destination_airport_iata|ID_aircraft|flight_ID|flight_number|on_ground|origin_airport_iata|number_registration_aircraft|
+-------------+------------+------------+------------------------+-----------+---------+-------------+---------+-------------------+----------------------------+
|         B752|          FX|         FDX|                     OAK|     ACDC63| 329cd5a3|       FX1802|        1|                RNO|                      N928FD|
|         B77L|          FX|         FDX|                     ANC|     ABC591| 329f5eb9|       FX5236|        1|                IND|                      N858FD|
|         A359|          DL|         DAL|                     ATL|     A67F8A| 32a11a70|        DL201|        0|                JNB|                      N518DZ|
|         A359|          DL|

In [20]:
#read df_airlines
display(df_airports)

DataFrame[country: string, airport_iata: string, airport_icao: string, airport_name: string]

In [21]:
display(df_zones)

DataFrame[Continent: string, Country: string]

In [22]:
# Filter flights in progress (on_ground is 0)
df_flights_in_progress = df_flights.filter(df_flights["on_ground"] == 0)
# join df_flights and df_airlins
df_join = df_flights_in_progress.join(df_airlines, on=[df_flights_in_progress.airline_icao ==df_airlines.airline_icao], how='left')
# Group by airline_iata and count, then order by count descending
df_most_flights_airline = (
    df_join.groupBy("airline_name")
    .agg(F.count("flight_ID").alias("num_flights"))
    .orderBy(F.desc("num_flights"))
).limit(1)

df_most_flights_airline.show()

+-------------+-----------+
| airline_name|num_flights|
+-------------+-----------+
|Qatar Airways|         85|
+-------------+-----------+



In [23]:
display(df_distance)

DataFrame[Flightnumber: string, Airport_icao: string, Distance: double]

## Pour chaque continent, la compagnie avec le + de vols régionaux actifs (continent d'origine == continent de destination)

In [24]:
#Convert the country columns in both dataframes to lowercase
from pyspark.sql.functions import lower

df_airports = df_airports.withColumn("Country", lower(df_airports["Country"]))
df_zones = df_zones.withColumn("Country", lower(df_zones["Country"]))

df1=df_airports\
    .join(df_zones, on=[df_airports.Country == df_zones.Country], how='left') \
    .join (df_distance, on=[df_distance.Airport_icao == df_airports.airport_icao], how='left')\
    .join (df_flights_in_progress,  on=[df_flights_in_progress.flight_number == df_distance.Flightnumber], how='right')


#drop columns duplicates
df_final= df1.drop(df_zones.Country, df_airports.airport_icao, df_distance.Flightnumber, )


In [25]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Filter out rows where continent is null
filtered_df = df_final.filter(df_final.Continent.isNotNull())

# Group by continent and airline_icao, and count the flights
grouped_df = (filtered_df.groupBy("Continent", "airline_icao")
             .agg(F.count("flight_number").alias("flight_count")))

# For each continent, order the airlines by their flight count in descending order
# and get the top airline
window_spec = (Window.partitionBy("Continent")
               .orderBy(F.desc("flight_count"), "airline_icao"))

top_airlines = (grouped_df.withColumn("rank", F.row_number().over(window_spec))
                .filter(F.col("rank") == 1)
                .drop("rank"))

top_airlines.show()

+---------+------------+------------+
|Continent|airline_icao|flight_count|
+---------+------------+------------+
|     asia|         QTR|        7140|
|   europe|         QTR|       34425|
+---------+------------+------------+



## Le vol en cours avec le trajet le plus long

In [26]:
#Join flight_progress with flight_airport_distance

df_flights_join = df_flights_in_progress.join(df_distance, on=[df_flights_in_progress.flight_number == df_distance.Flightnumber ], how='left')
df_flight_distance=df_flights_join.select("flight_number", "Distance").orderBy(F.desc("Distance")).limit(1)

display(df_flight_distance)

DataFrame[flight_number: string, Distance: double]

## Pour chaque continent, la longueur de vol moyenne

In [27]:
df_avg= df_final.select("continent", "flight_number","Distance")

from pyspark.sql import functions as F

# Filter out rows where continent is null
filtered_df = df_avg.filter(df_avg.continent.isNotNull())

# Group by continent and compute the average distance
avg_distance_df = (filtered_df.groupBy("continent")
                   .agg(F.avg("Distance").alias("average_distance")))

avg_distance_df.show()

+---------+------------------+
|continent|  average_distance|
+---------+------------------+
|   europe|5922.6780874515825|
|     asia| 8168.011560421713|
+---------+------------------+



##  Pour chaque pays de compagnie aérienne, le top 3 des modèles d'avion en usage

In [28]:
df_airlines_air= df_final.select("aircraft_code", "flight_number","Country","airline_icao")
display(df_airlines_air)

df_airlines_aircraft= df_airlines_air.join(df_airlines, "airline_icao", how='left')

from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Group by airline_name, country and aircraft_code and count
df_grouped = df_airlines_aircraft.groupBy("airline_name", "aircraft_code", "country").agg(F.count("flight_number").alias("count"))

# Define a window specification to partition by both airline_name and country
window_spec = Window.partitionBy("airline_name", "country").orderBy(F.desc("count"))

# Use the rank function over the window specification
df_grouped = df_grouped.withColumn("rank", F.rank().over(window_spec))

# Filter for top 3 models for each airline within each country
df_top3 = df_grouped.filter(F.col("rank") <= 3).drop("rank", "count")



df_filter = df_top3.filter(df_top3.airline_name.isNotNull())
df_filter.show()

DataFrame[aircraft_code: string, flight_number: string, Country: string, airline_icao: string]

+------------+-------------+-------------------+
|airline_name|aircraft_code|            country|
+------------+-------------+-------------------+
|AMC Airlines|         B738|        afghanistan|
|AMC Airlines|         B738|            albania|
|AMC Airlines|         B738|           anguilla|
|AMC Airlines|         B738|         antarctica|
|AMC Airlines|         B738|antigua and barbuda|
|AMC Airlines|         B738|          argentina|
|AMC Airlines|         B738|            armenia|
|AMC Airlines|         B738|              aruba|
|AMC Airlines|         B738|          australia|
|AMC Airlines|         B738|             bhutan|
|AMC Airlines|         B738|            burundi|
|AMC Airlines|         B738|             canada|
|AMC Airlines|         B738|         cape verde|
|AMC Airlines|         B738|           colombia|
|AMC Airlines|         B738|            croatia|
|AMC Airlines|         B738|               cuba|
|AMC Airlines|         B738|             cyprus|
|AMC Airlines|      

## L'entreprise constructeur d'avions avec le plus de vols actifs

In [29]:
df_airlines_air= df_final.select("aircraft_code", "flight_number","Country","airline_icao")
display(df_airlines_air)

df_airlines_aircraft= df_airlines_air.join(df_airlines, "airline_icao", how='left')

DataFrame[aircraft_code: string, flight_number: string, Country: string, airline_icao: string]

In [30]:
from itertools import chain


    # Dictionnaire des modèles d'avions et leurs fabricants respectifs
manufacturers = {
        "CL35": "Bombardier",
        "A21N": "Airbus",
        "B742": "Boeing",
        "A332": "Airbus",
        "B772": "Boeing",
        "GLF6": "Gulfstream",
        "E295": "Embraer",
        "B78X": "Boeing",
        "A319": "Airbus",
        "A343": "Airbus",
        "A320": "Airbus",
        "C750": "Cessna",
        "C68A": "Cessna",
        "A388": "Airbus",
        "B744": "Boeing",
        "A321": "Airbus",
        "GLEX": "Bombardier",
        "B789": "Boeing",
        "B748": "Boeing",
        "BCS3": "Airbus",
        "A333": "Airbus",
        "B738": "Boeing",
        "B763": "Boeing",
        "T204": "Tupolev",
        "GL7T": "Gulfstream",
        "B788": "Boeing",
        "MD11": "McDonnell Douglas",
        "0000": "Unknown", # Ce code semble être une valeur inconnue ou un espace réservé
        "B737": "Boeing",
        "A310": "Airbus",
        "A339": "Airbus",
        "C680": "Cessna",
        "B752": "Boeing",
        "A35K": "Airbus",
        "A346": "Airbus",
        "B77W": "Boeing",
        "SU95": "Sukhoi",
        "C56X": "Cessna",
        "A359": "Airbus",
        "H25B": "Hawker Beechcraft",
        "A20N": "Airbus",
        "B77L": "Boeing",
        "B38M": "Boeing",
        "B739": "Boeing",
        "B39M": "Boeing"
    }
    
# Convertir le dictionnaire Python en un dictionnaire Spark
mapping_expr = create_map([lit(x) for x in chain(*manufacturers.items())])

# Add column  'manufacturer'
df = df_airlines_aircraft.withColumn("manufacturer", mapping_expr[df_airlines_aircraft["aircraft_code"]])

# Grouper par fabricant et compter
df_manufacturer_counts = df.groupBy("manufacturer").count().orderBy("count", ascending=False)

# Afficher le fabricant avec le plus de vols
df_manufacturer_counts.show(1)


+------------+-------+
|manufacturer|  count|
+------------+-------+
|      Boeing|4253915|
+------------+-------+
only showing top 1 row

